In [1]:
! pip install datasets transformers nltk evaluate sacrebleu ipdb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 50.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behavi

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
import nltk
nltk.download('punkt_tab')
import transformers

from IPython.core import error
import json
from fnmatch import fnmatchcase as match
import random
import os
import pandas as pd
import datasets
from datasets import Dataset, DatasetDict, load_dataset, load_from_disk
import numpy as np
from argparse import ArgumentParser
from typing import Dict, List, Union
import ast
import csv
import ipdb
import ast
import copy
import shutil

## params
int_seed = 1203
dataset_name = "LTL_koreauniv"
init_weight = "with_pre-train"
data_size = "0.1-0.9"
model_checkpoint = "t5-base"
print(model_checkpoint)
print('\n')

home_path = "/content/drive/MyDrive/Colab_Notebooks/github/NL2TL"
data_dir = "dataset"
data_filename = "command_LTL_dataset_v01.csv"
data_augment_filename = "aug.csv"
data_path = os.path.join(home_path, data_dir, data_filename)
data_augment_path = os.path.join(home_path, data_dir, data_augment_filename)

symbol_to_word_map = {
    "F(": "finally(",
    "G(": "globally(",
    "U(": "until(",
    "&": "and",
    "¬": "not",
    "∧": "and",
    "∨": "or",
    "→": "imply",
}

# read original csv
with open(data_path, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    data = []
    for row in reader:
        data.append([ast.literal_eval(row["generated_command"]),
                        ast.literal_eval(row["generated_LTL"])])

# align dataset format and write to csv
align_data_filepath = os.path.join(home_path, data_dir, data_filename[:-4] + "_realign.csv")
align_data = []
with open(align_data_filepath, 'w') as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(["id", "ltl", "sentence"])
    i = 1
    for item in data:
        sents, ltls = item[0], item[1]
        for s in sents:
            for l in ltls:
                # convert symbol to word
                for k, v in symbol_to_word_map.items():
                    l = l.replace(k, v)
                csv_writer.writerow([i, l, s])
                align_data.append([i, l, s])
                i += 1
align_data_num = i - 1

# augment data and write csv
align_aug_data_filepath = align_data_filepath[:-4] + "_aug.csv"
aug_filepath = os.path.join(home_path, data_dir, "aug.csv")
shutil.copy(align_data_filepath, align_aug_data_filepath)
with open(align_aug_data_filepath, 'a') as f:
    csv_writer = csv.writer(f)
    with open(aug_filepath, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            ltl_obj, com_obj = row["ltl_obj"], row["com_obj"]
            aug_ltl_obj = ast.literal_eval(row["ltl_obj_aug"])
            aug_com_obj = ast.literal_eval(row["com_obj_aug"])
            for ad in align_data:
                if ltl_obj in ad[1] and com_obj in ad[2]:
                    for aug_l, aug_c in zip(aug_ltl_obj, aug_com_obj):
                        csv_writer.writerow([i, ad[1].replace(ltl_obj, aug_l), ad[2].replace(com_obj, aug_c)])
                        i += 1
align_aug_data_num = i - 1

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


t5-base




In [4]:
from transformers import AutoTokenizer
import evaluate

# path
input_model_dir = os.path.join(home_path, "model", "t5-base-epch20-infix-word-04-21", "checkpoint-62500")
output_model_dir = os.path.join(home_path, "model", "t5-base-transfer-learning/")
if not os.path.exists(output_model_dir):
  os.mkdir(output_model_dir)

# tokenizer and params
tokenizer = AutoTokenizer.from_pretrained(input_model_dir)
max_input_length = 1024
max_target_length = 128
prefix = "Transform the following sentence into Signal Temporal logic: "

# preprocess data
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["sentence"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["ltl"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    model_inputs["sentence"] = examples["sentence"]
    model_inputs["ltl"] = examples["ltl"]
    model_inputs["id"] = examples["id"]
    return model_inputs

# define compute metrics
bleu_metric = evaluate.load("bleu", force_prefix=True)
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # print(predictions)
    # print(labels)
    # Replace -100 in the labels as we can't decode them.
    predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    count = 0

    # top-1 accuracy
    for i in range(len(decoded_preds)):
        pred = nltk.sent_tokenize(decoded_preds[i].strip())
        label = nltk.sent_tokenize(decoded_labels[i].strip())
        if pred == label:
            count += 1

    bleu = bleu_metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {'top-1 accuracy': round(count / len(decoded_preds), 6),
            'bleu score': bleu['bleu'],
            'bleu precisions': bleu['precisions']}

# correct parenthesis
def correct_parenthe(input_str):
  count = 0
  original_list = input_str.split(' ')
  for index, item in enumerate(original_list):
    if len(item) >2:
      if item[-1] == '.':
        original_list[index] = original_list[index][:-1]
    if item == '(':
      count += 1
    elif item == ')':
      count -= 1
  if count >0:
    for i in range(count):
      original_list.append(')')
  if count <0:
    for i in range(-count):
      original_list.pop(-1)
  return ' '.join(original_list)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The below is the training code block. So you would need to run the below for transfer-learning fit to the specific dataset.

In [ ]:
# transfer learning
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

# load realigned dataset
AUGMENT_DATA = True
dataset = load_dataset('csv', data_files=align_aug_data_filepath if AUGMENT_DATA else align_data_filepath)["train"]
train_dataset, test_dataset = dataset.train_test_split(test_size=0.1, shuffle=True, seed=int_seed).values()
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True)

# load pretrained model
model = AutoModelForSeq2SeqLM.from_pretrained(input_model_dir)
batch_size = 16
output_model_name = model_checkpoint.split("/")[-1]+'-'+dataset_name+"-epoch20-trainpoint"+ \
                    str(align_aug_data_num) if AUGMENT_DATA else str(align_data_num)
output_model_dir = output_model_dir+output_model_name

# set trainer params
args = Seq2SeqTrainingArguments(
    output_model_dir,
    output_model_name,
    evaluation_strategy = "steps",
    eval_steps=1000,
    logging_steps=1000,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    seed=int_seed,
    save_total_limit=1,
    num_train_epochs=20,
    predict_with_generate=True,
    fp16=False,
    #push_to_hub=True,
    #report_to="tensorboard",
    #load_best_model_at_end=True,
    #save_strategy = "no"
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# instantiate trainer
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_train_dataset ,
    eval_dataset=tokenized_test_dataset ,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# run train
trainer.train()
trainer.save_model()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-6-36447d5cea17>:43: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss,Top-1 accuracy,Bleu score,Bleu precisions
1000,0.151700,0.001714,0.272300,0.382877,"[0.9312436804853387, 0.9177489177489178, 0.9002364066193853, 0.8774422735346359]"
2000,0.006200,0.001354,0.274648,0.386235,"[0.9334006054490414, 0.9218688653317628, 0.9061763319189062, 0.8831858407079646]"
3000,0.003100,0.000954,0.274648,0.386235,"[0.9334006054490414, 0.9218688653317628, 0.9061763319189062, 0.8831858407079646]"
4000,0.001900,0.001036,0.274648,0.386235,"[0.9334006054490414, 0.9218688653317628, 0.9061763319189062, 0.8831858407079646]"


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Trainer is attempting to log a value of "[0.9312436804853387, 0.9177489177489178, 0.9002364066193853, 0.8774422735346359]" of type <class 'list'> for key "eval/bleu precisions" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Trainer is attempting to log a value of "[0.9334006054490414, 0.9218688653317628, 0.9061763319189062, 0.8831858407079646]" of type <class 'list'> for 

The below code block is for testing using the above transfer-learned model or already pre-trained model. If you want to only test without training, then run from the first code block but skip the just previous training code block and run the below code block.

In [5]:
# Test with a transfer learning model
import torch
import evaluate
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from functools import reduce

int_seed = 1203
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
show_diff_parenthe_results = False

# transfer learning model with 2082 dataset
# output_model_dir = os.path.join(home_path, "model", "t5-base-transfer-learning",
#                                f"t5-base-LTL_koreauniv-epoch20-trainpoint2082", "checkpoint-2340")

# transfer learning model with 33080 dataset with augmentation
output_model_dir = os.path.join(home_path, "model", "t5-base-transfer-learning",
                                  f"t5-base-LTL_koreauniv-epoch20-trainpoint33080", "checkpoint-28500")

# pre-trained model
# output_model_dir = os.path.join(home_path, "model", "t5-base-epch20-infix-word-04-21", "checkpoint-62500")

model = AutoModelForSeq2SeqLM.from_pretrained(output_model_dir).to(device)
acc_top1 = 0.
acc_bleu_scores = 0.
acc_bleu_precisions = [0., 0., 0., 0.]

metric = evaluate.load("accuracy")
dataset = load_dataset('csv', data_files=align_data_filepath)['train']
train_dataset, test_dataset = dataset.train_test_split(test_size=0.1, shuffle=True, seed=int_seed).values()
tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True)
prefix = "Transform the following sentence into Signal Temporal logic: "

with open(output_model_dir +'/result.txt', 'w') as f_result:
    for i in range(len(tokenized_test_dataset)):
        input = [prefix + tokenized_test_dataset[i]['sentence']]
        f_result.write("input:\n")
        f_result.write(f"{input[0]}\n")

        input = tokenizer(input, max_length=max_input_length, truncation=True, return_tensors="pt").to(device)
        output = model.generate(**input, num_beams=8, do_sample=True, min_length=10, max_length=64)
        decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
        label = tokenized_test_dataset[i]['ltl']
        predicted_output = correct_parenthe(decoded_output.strip())

        multi_labels = [t["ltl"] for t in dataset if test_dataset[i]["sentence"] == t["sentence"]]
        if predicted_output in multi_labels:
            acc_top1 += 1
        #if predicted_output == label:
        #    acc_top1 += 1

        f_result.write(f"label:\n")
        for l in multi_labels:
            f_result.write(f"{l}\n")
        f_result.write(f"pred:\n")
        f_result.write(f"{predicted_output}\n")
        f_result.write("\n")

        bleus = [bleu_metric.compute(predictions=[predicted_output], references=[ml]) for ml in multi_labels]
        acc_bleu_scores += reduce(lambda x, y: max(x, y), [bs["bleu"] for bs in bleus])
        acc_bleu_precisions = [acc_bleu_precisions[n] + j for n, j in enumerate(reduce(lambda x, y: max(x, y), [bs["precisions"] for bs in bleus]))]

        #bleu = bleu_metric.compute(predictions=[predicted_output], references=[label])
        #acc_bleu_scores += bleu["bleu"]
        #acc_bleu_precisions = [acc_bleu_precisions[n] + j for n, j in enumerate(bleu["precisions"])]

    test_data_num = i + 1
    acc_top1 /= test_data_num
    acc_bleu_scores /= test_data_num
    acc_bleu_precisions = [j / test_data_num for j in acc_bleu_precisions]

    print(f"The test data number = {test_data_num}")
    print(f"Top-1 accuracy = {acc_top1}")
    print(f"Bleu score = {acc_bleu_scores}")
    print(f"Bleu precision = {acc_bleu_precisions}")
    f_result.write('\n')
    f_result.write(f"The test data number = {test_data_num}\n")
    f_result.write(f"Top-1 accuracy = {acc_top1}\n")
    f_result.write(f"Bleu score = {acc_bleu_scores}\n")
    f_result.write(f"Bleu precision = {acc_bleu_precisions}\n")

f_result.close()

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/208 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3953: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


The test data number = 208
Top-1 accuracy = 0.9855769230769231
Bleu score = 0.9855769230769231
Bleu precision = [0.9913461538461539, 0.9903846153846154, 0.9855769230769231, 0.9855769230769231]
